<!-- autodiff documentation master file, created by
sphinx-quickstart on Tue Nov  9 21:28:25 2021.
You can adapt this file completely to your liking, but it should at least
contain the root `toctree` directive. -->
# Welcome to Autodiff’s Documentation!

### Extended Documentation:

* node module
* Dualnumber module
* Operatorsfunc module


# Introduction

Differentiation is an important operation in scientific research broadly
applied in non-linear optimization problems and numerical solution of
differential equations. Recently, the development of machine learning
techniques including artificial neural networks also require large amount of
gradient computation. For complex functions whose analytical derivative
expressions are difficult to obtain, one traditional option to calculate its
derivative is finite-difference method, but may have issues such as
floating point errors. Automatic differentiation (AD) refers to a general way
of computing the derivatives of functions expressed as computation graphs, with
which the evaluation of derivatives could reach machine precision. Our autodiff
package provides tools to calculate the derivative of any function with an
analytic expression based on the AD technique.

# Background

AD mainly makes use of two properties of closed-form functions:
1. Any closed-form function is a combination (sum, product, quotient or compound) of
elementary functions, and the analytical expressions of the derivatives of
elementary functions are already-known.
2. Derivative rules including sum rule, product rule, quotient rule and most importantly, the chain rule.
- sum rule: $(f+g)’ = f’+g’$
- product rule: $(f\times g)’ = f’g+g’f\times g$
- quotient rule: $(f/g)’ = (f’g-g’f)/(g\times g)$
- chain rule: $h = f(g(•))$, then $h’ = f’(g(•))× g’(•)$

To evaluate the value of a complicated function expression, we need to start with
independent variables and evaluate a series of intermediate results and finally
reach the final result. The idea of AD is to use a computational graph to
represent the function where each node represents an intermediate variable and
only elementary operations are carried out between intermediate variables. Because we
utilize the operation rules of derivatives mentioned above, the derivative at any
node in the computation graph can only rely on its parent node(s). As a result, we
obtain the final derivative result by computing derivatives and repeatedly utilizing
derivative rules following the flow of the computation graph.

Here we give a brief example to illustrate how we calculate derivatives 
based on compution graph. If we want to evaluate the derivative with respect 
to x1 at x1=pi/2 for function f=sin(cos(x1)), here is our computation graph

![comp graph](fig1.png)

Then we follow the graph and evaluate step by step:

|step| trace | Elementary function |Current value | Elementary function derivative| $\nabla_x$ value|
|--|--|--|--|-- |--|
|1|v0|x|pi/2|$\dot x$|1|
|2|v1|cos(v0)|0|$-sin(v_0)\dot v_0$|0|
|3|f|sin(v1)|0|$cos(v_1)\dot v_1$|0|

Note that when we calculate "elementary function derivative", we made use of the chain rule. 

# How to use

## Installation

### Option 1:


### Distribution 
We will used PyPI to distribute our package.   
We will used a library called `twine` to help us package and distribute our
package over PyPI. We will follow this helpful tutorial
https://packaging.python.org/tutorials/packaging-projects/, provides a detailed
guide on how to setup the structure. Using a framework may be helpful,
but in an effort to reduce software overhead and maximize learning for each
step, we will elect to forgo a framework. Users may install autodiff (?? is this the name) through the following command line:

```
pip3 install autodiff--aabj
```

Within a python terminal, you can then import the package:

```
import autodiff
```

Alternatively, to pull commmon functions directly you may use the following import statement:

```
from autodiff.node import variables, sin, cos, tan, exp, ln, log, sinh, cosh, tanh, sqrt, logistic, log_ab
```

### Option 2:

In adition to pip installation, you may clone directly from the repository:

```
git clone https://github.com/cs107-sweet-16/cs107-FinalProject.git
```

After successful cloning of the repository, follow by installing the package requirements from the ‘requirements.txt’ file.

```
cd cs107-FinalProject
pip3 install -r requirements.txt
```

Finally, finish by importing dualnumber and it’s operators

```
import numpy as np
from node import sin, cos, tan, exp, log, valNode
```



## Interaction

Autodiff executes auto-differntiation by utilizting the concept of a computational graph. This graph is constructed during the execution of a forward pass. Simultaneously, the final value, and the respective gradient of each variable are also computed. Forward mode may be passed on some simple user-defined function, e.g. $$ f = 3x + 5^y $$. The computational graph is constructed via Python's native operator precedence and an ordered dictionaries. As each step is executed per Python's operator precedence, the respective values and derivatives are stored within the dictionary. The code for this may be found under the directory `autodiff/`

### Workflow

The general workflow for this project consists of the following:
1. Determining some function of interest
2. Initializing variables
3. Defining the function
4. Setting the variable values
5. Executing forward mode or reverse mode to calculate the value of the function and derivatives
The order of Step 3. and Step 4. can be switched, but the values of the variables must be set before the function is evaluated and differentiated.


### Multivariate Scalar Function Example

When we use multiple variables to construct functions, we first declare the variables with `variable(name, size=None)` function, which accepts `name` as the names to identify the variables. When we declare scalar variables, we should not pass the `size` argument, or set `size=1`.


In [13]:
import numpy as np
from autodiff.node import variables, sin, cos, tan, exp, ln, log, sinh, cosh, tanh, sqrt, logistic, log_ab, vector

# EXAMPLE 1 for forward mode

# testing cos(ab/c) + c*log(a), a = 4, b = -1, c = 10

# initialize the nodes as variables"
a = variables('a')
b = variables('b')
c = variables('c')



Then we use the python operands or the elementary functions to construct the function expression, and set the values of the variables with `.set_val(val)` method. 


In [15]:
# define the function:
f = cos((a * b) / c) + c * log(a)

# set the values of variables
a.set_val(4)
b.set_val(-1)
c.set_val(10)

Then, we can use `.forward()` to evaluate the function and calculates its derivatives in forward-mode automatic differentiation. 

In [16]:
# execute forward:
f_val, f_grad = f.forward()


# print the values of interest:
print("value of function f: ", f_val)
print("gradient with respect to a: ", f_grad['a'])
print("gradient with respect to b: ", f_grad['b'])
print("gradient with respect to c: ", f_grad['c'])

value of function f:  14.78400460520179
gradient with respect to a:  2.461058165769135
gradient with respect to b:  0.15576733692346023
gradient with respect to c:  1.4018710948122366


Or we can use `.reverse()` instead to calculte the function value and derivatives in reverse-mode automatic differentiation.

In [17]:
# EXAMPLE 2 for reverse mode

# execute reverse:
f_val, f_grad = f.reverse()


# print the values of interest:
print("value of function f: ", f_val)
print("gradient with respect to a: ", f_grad['a'])
print("gradient with respect to b: ", f_grad['b'])
print("gradient with respect to c: ", f_grad['c'])

value of function f:  14.78400460520179
gradient with respect to a:  2.461058165769135
gradient with respect to b:  0.15576733692346023
gradient with respect to c:  1.4018710948122366


### Vector Function and Vector Input Example

We can also define vector inputs and vector functions with `autodiff` package. To define a vector variable, call `variable(name, size)` with the name of the vector and the size of the vector variable. We can set the values of the vector variable by passing a list of values to `.set_val()` method.

In [19]:
## EXAMPLE 3 for vector function and vector input:
# declare a as a vector
a = variables('a', 3)
# set a's values
a.set_val([1,2,3])

 To define a vector function, we need to use `vector` to vectorize all the output elements as a `vector` instance.

In [20]:
# set functions:
# f1(a) = a[0]**2
# f2(a) = a[1]*a[2]
# f = [f1, f2]
# the output function need to be returned as a vector
def func(a):
    f1 = a[0]**2
    f2 = a[1]*a[2]
    return vector(f1, f2)
f = func(a)

After the function is defined and constructed, we can use `.evaluate()` to evaluate the function and use `.grad()` to calculate the Jacobian matrix.

In [21]:
# calculate function values
f_val = f.evaluate()

# calculate Jacobian matrix of the function
f_grad = f.grad(a)

# print the values of interest:
print("value of function f: ", f_val)
print("gradient of a at a=[1,2,3]: ", f_grad)

value of function f:  [1, 6]
gradient of a at a=[1,2,3]:  [[2, 0, 0], [0, 3, 2]]



### Simple Usage with Dual Numbers

In addition to our node.py file, which contains code for forward and reverse passes on a given function, we also have more rudimentary code of `Dualnumber` class that constructs dual numbers and use forward-mode automatic differentiation to calculate the value and derivatives of functions. This `Dualnumber` class is more convenient to use when we have a single scalar input and scalar function. The files containing `Dualnumber` class are: `Dualnumber.py` and `Operatorsfunc.py`. Both of which can be found along within the `autodiff/` diretory.

We can initialize a variable with `Dualnumber(val)`, which accepts an argument to be the value of the variable. 

In [22]:
import numpy as np
from autodiff.dualnumber import Dualnumber
from autodiff.operatorsfunc import tan, exp
# set dual number 'x'
# first value (np.pi) is the real part at which we want to calculate the derivative and value of the function
# second value '1' is the dual part. In this case it is set to one, and will default to 1, but there is an option to change it. 
x = Dualnumber(np.pi, der=1)


Next, implement the function by substituting in the defined dual number variable. The value and derivative of the function is calculated during this process, and can be retrived by `.val` and `.der`.

In [23]:

# Next implement your function, by substituting in the defined dual number:
f = (tan(x)) - 2 ** x * exp(x)

# The forward pass has now been executed and we can check the value of the function and it's derivative:

print("value of function f: ", f.val)
print("gradient of x at x = pi: ", f.der)

value of function f:  -204.2160993874646
gradient of x at x = pi:  -344.76791290283523


## Software Organization

### Directory Structure
Our code will live in a separate folder called `src`. We will have a separate
directory, `test`, for our tests. Finally, while not necessarily code, our
documentation will live in a directory named `docs`, which will go into
technical detail how each user-facing function is used, and any examples if
appropriate.

```
CS107-FinalProject/
  autodiff/
    Dualnumber.py
    Operatorsfunc.py
    node.py
  docs/
    source/
      ...
    documentation.md
    milestone1.md
    milestone2.md
    milestone2_progress.md
  test/
    __init__.py
    test_node.py
    test_dual.py
    test_node_errs.py
    test_operatorsfunc.py
```

### Modules

We will have one package, named `autodiff`. Within this module there are a
few modules, with the most core functionality in a module named `node.py`.  This
module will provides us with the basic functionality of being able to
differentiate on a defined set of elementary operations and functions. 

In addition to `node.py`, we also have `Dualnumber.py` and `Operatorsfunc.py` which has our dated implementation of autodiff in them. `Dualnumber.py` constructs a Dualnumber class, with native Python functions. `Operatorsfunc.py` consists of extended functions, such as sinusoidial, to extend the functionality of `Daulnumber.py`. 

### Test Suite

Our test suite lives in a separate directory called `tests`. We will use
TravisCI to run continuous integration, and make sure we do not accidentally
introduce regressions in our code when we change or add functionality. CodeCov
is used to ensure that any new code that we write is properly tested and
accounted for.

### Documentation
We will use Sphinx as our documentation generator. To build sphinx, you can run the command inside
the `docs/` directory:
```
sphinx-build source/ build/
```
The generated HTML files are viewable as `cs107-FinalProject/docs/build/index.html`.
    






## Implementation 
We have a few core data structures:
* Dual numbers
* Binary-tree based computational graph

### Dual Number for Simple Usage





We first use dual number to implement basic forward mode for single variable scalar inputs. A dual number consists of a real part and a *dual* part, which is written as
$$ z=a+b\epsilon $$
where $a,b\in \mathbb{R}$ and $\epsilon$ is a special number such that $\epsilon^2=0$ and $\epsilon\neq 0$.We notice that dual numbers have the following property
$$ f(a+b\epsilon) = f(a) + (f’(a)\times b)\epsilon $$
For a univariate scalar function, it is convenient to use dual numbers to calculate its derivative this way. We overload python dunder methods for basic operands and defined other basic arithmetic functions with the `Dualnumber` class to automate this calculation process.


#### class Dualnumber(a, der=1)
We will begin by constructing a dualNumber() class that
deals with the properties of dual numbers. It will take a real value and dual
value as the initial input. The returns of calculations with dual numbers are also return dual numbers. Real numbers are considered as dual numbers with dual part=0.
```py
class Dualnumber:
    def __init__(self, a, der=1):
        self.val = a
        self.der = der

    def set_dual(self, dual):
        self.der = dual

    # dunder methods ...
```

#### Primitives
The `Dualnumber` class supports the following operations and functions:
* `+`(`__add__`,`__radd__`): add
* `-`(`__sub__`,`__rsub__`): subtract
* `*`(`__mul__`,`__rmul__`): multiply
* `/`(`__truediv__`,`__rtruediv__`): truedivide
* `**`(`__pow__`,`__rpow__`): power function
* `-`(`__neg__`): standard unary negative operation
* `+`(`__pos__`): standard unary positive operation
* `sin`: sine function
* `cos`: cosine function
* `tan`: tangent function
* `exp`: natural exponent
* `log`: logarithm base e

### Computational Graph
#### Data Structure
We build our computational graph based on binary tree data structure. The basic element of the graph is the node, which is represented as `Node` class in implementation. Each node in the graph can have at most two children.

#### class node.Node()
Bases: `object`

`Node` is the basic element that constitutes the computational graph. `Node` supports unary and binary elementary functions, including native python functions such as `+-*/`, and elementary arithmetic functions such as `sin`, `cos` (all the elementary functions supported by `Node` is listed in *Primitive* section below). When an elementary function is called on `Node` instances, it will generate a new `Node` as the parent of the input `Node` instances, record the function or operation that should be implemented on the two input `Node`s in the new `Node`, and return the new `Node`. If a `Node` is calculated with a constant `float` or `int` input, the constant will be converted to a constant node in the computational graph. The returned `Node` can be considered as an intermediate variable in the computational graph. Therefore, during the process of using elementary functions on `Node`s to construct the complex function expression we want to calculate, we obtain a computational graph representing this function.

#### class node.valNode(name=None)
Bases: `node.Node`

A `valNode` instance represents an input variable with respect to which we want to calculate derivatives, or a constant input. `valNode` should not have any child in the computational graph.
* `valNode.name`: the name of the variable, which is used to distinguish different variables. `valNode.name` being `None` means it represents a constant input.
* `valNode.val`: the value set for the variable, or the value of the constant.

#### class node.funcNode(func, leftdf, rightdf)
Bases: `node.Node`

A `funcNode` instance represents an function that should be executed on one or two (independent or intermediate) variables in the computational graph. `funcNode` can represents both unary and binary functions.
* `funcNode.left`: the first variable to be passed into the function
* `funcNode.right`: the second variable to be passed into the function, should be `None` when the `funcNode` represents a unary function
* `funcNode.func`: the arithmetic function to be implemented on the value of children nodes, could be either unary or binary
* `funcNode.leftdf`: the arithmetic function which accepts the value of two variables and calculate the derivative of `funcNode.func` with respect to the first variable, should accept only one value if `funcNode.func` is a unary functio
* `funcNode.rightdf`: the arithmetic function which accepts the value of two variables and calculate the derivative of `funcNode.func` with respect to the second variable, should be `None` if `funcNode.func` is a unary function


#### Forward Mode
The chain rule for forward-mode automatic differentiation is
$$ v_3 = f(v_1, v_2) $$
$$  \nabla v_3 = \frac{\partial f}{\partial v_1}\nabla v_1 + \frac{\partial f}{\partial v_2}\nabla v_2  $$
$v_1$, $v_2$, $\nabla v_1$, and $\nabla v_2$ must be calculated before calculating $v_3$ and $\nabla v_3$. 
* `funcNode.forward()`: Executes a forward pass automatic differentiation. Recursively calculate the values and derivatives of left and right chidren, and use the calculation results to calculate self value and self derivatives. Keep track of independent variables with dictionary and the names of independent variables.

#### Primitives
Node also contains elementary operations. 
We will maintain an updated list of these operations:

* `+`(`__add__`,`__radd__`): add
* `-`(`__sub__`,`__rsub__`): subtract
* `*`(`__mul__`,`__rmul__`): multiply
* `/`(`__truediv__`,`__rtruediv__`): truedivide
* `**`(`__pow__`,`__rpow__`): power function
* `-`(`__neg__`): standard unary negative operation
* `+`(`__pos__`): standard unary positive operation
* `sin`: sine function
* `cos`: cosine function
* `tan`: tangent function
* `sinh`: hyperbolic sine function
* `cosh`: hyperbolic cosine function
* `tanh`: hyperbolic tangent function
* `arccos`: inverse cosine function
* `arcsin`: inverse sine function
* `arctan`: inverse tangent function
* `exp`: natural exponent
* `ln`: logarithm base e
* `log_ab`: logarithm with arbitary base
* `sqrt`: square root



### Dependendies

Our primary dependency within our working modules is `numpy`. `Numpy` was used to calculate values for non-native elementary functions, such as the sinusoidal classes. However we used numerous packages throughout our project. These include PyPi to form our project into a package, and codecove to determine code coverage.


## Extension: Reverse Mode
Besides forward-mode automatic differentiation supporting multiple input variables, we also implemented reverse-mode automatic differentiation with our computational graph structure. The reverse mode, similar to the forward mode, is also based on chain rule of derivatives, but it is more efficient than the the forward mode when the number of input variables are more than the number of outputs.

### Implementation details
The chain rule for reverse-mode automatic differentiation is
$$\bar{v_i} = \frac{\partial J}{\partial v_i} = \sum_{i\ \mathrm{a\ child\ of}\ j} \frac{\partial v_j}{\partial v_i} \frac{\partial J}{\partial v_j} = \sum_{i\ \mathrm{a\ child\ of}\ j} \frac{\partial v_j}{\partial v_i} \bar{v_j}$$
where $\bar{v_i}= \frac{\partial J}{\partial v_i}$ is the adjoint of $v_i$. The adjoint of an independent variable is equivalent to the derivative of $J$ with respect to this variable. In the reverse mode, the value of all intermediate variables need to be calculated before any derivatives and adjoints are calculated. The adjoints $\bar{v_i}$ and partial derivatives $\frac{\partial v_j}{\partial v_i}$ are passed top-to-bottom (from root to leaf) in the computational graph.

* `funcNode.forward_pass()`: Execute a forward pass to calculate the variables of all intermediate variables in the computational graph.
* `funcNode.reverse_pass()`: Executes a reverse pass to calculate the adjoints of all intermediate variables and independent variables. Recursively pass partial derivatives and adjoints to the children nodes.

### Additional attributes
* `funcNode.val`: store the value of intermediate variables calculated during the forward pass
* `valNode.der`: store the adjoint of the `valNode` (i.e., the derivative) calculated during the reverse pass


## Broader Impact and Inclusivity Statement

### Broader Impact

Automatic differentiation is used in many areas of scientific research and engineering. While autodifferentiation itself is not a new topic (and has been implemented many times), creating and building a autodifferentiation library is useful for understanding the concepts and elucidates how ubiquitously it can be used. Additionally, maintaining an open source library is good practice for understanding how to serve the software development community - writing clean, performant code, adhering to software development best practices, and being open to consider future extensions and migration of code standards.

As in all math, part of the nature of autodifferentiation is that it can be used in both socially good and harmful ways. There are real-world consequences, for example, of using autodifferentiation to train a machine learning model for a specific purpose, particularly on user privacy. To that end, we encourage user feedback on the usage of the our library, and to open a conversation on its potential misuse.

### Inclusitivity

The authors of the autodiff library welcome everyone and encourage participation and discussion of how to use autodifferentiation, regardless of any social status, standing, or nature. These include (but are not limited to):

age, culture, ethnicity, gender, natioanlity, politics, race, sex, sexual orientation, socioeconomic status This community fosters mutual respect and tolerance - while we cannot hope to be knowledgable about everything, we can hope to be better, and more informed than when we began. Our autodiff library is not meant to be rigid body, but rather an evolving piece of software that changes along with its community.

Both contributors and core developers of the library are expected to hold each other accountable, and to open conversation with optimism, and not of expectation. To this end, we will have pull requests and issues reviewed in a thorough and unassuming manner.

We encourage any and all underrepresented groups interested in software or learning more about how our code works to submit thoughtful tickets, raise issues, and ask for feature requests to help increase the inclusivity of our wording, and make it more accessible to experience and inexperienced Python practitioners alike, native and non-native English speakers as well.

## Future

### Improving API
In future, we would like to incorporate improved API for vector inputs. Currently, the user needs to define
all functions for vector inputs. While this can't be done away in cases where there are multiple independent output functions f: $R^{n}$  → $R^{m}$, we can improve the user interface for cases where the output function is same for each of the output dimentions m. For example, currently calulating `sin(v)` for vector input `v` requires user to define a function for each output nodes which are all sine function. A better improved API could do away with that requirement.

### Higher-Order Derivatives
We would also like to support the calculation of ***Hessian*** function. Hessian, the second order derivative of a function f : $R^{n}$ → R, is a n x n square matrix **$(H_{f})_{i, j}$** = $\dfrac{\partial^2 f}{\partial x_i\partial x_j}$

Hessian has many important application across various problems in science and engineering. For example, it is used in approximating functions as polynomial with higher precision in the Taylor Series expansion. Several interesting mathemetical property of functions like infelection points and determination of function curvature require calculating the second derivative of the function. Many optimaization functions in deep learning (for example, Newton's method) require computing Hessian function.

To add Hessian function support in our package, we would need to change the existing implementation of reverse mode to support computational graph as an input to it. Hessian can be then calculated by a reverse mode applied on the compuational graph of the gradient calculate in the forward mode. In essence, this is equivalent to forward mode computation followed by reverse mode. Implementing Hessian is computationally expensive in terms of time and space requiremnts ($O(n^2)$ in both.) There are numerical methods to compute approximations for Hessians functions using quasi-Newton methods, that are more efficient.
